# Power Capacity Expansion Problem

Install packages, setup Gurobi license and read in data. $\color{red}{\text{Add the codes for your license.}}$

In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd
import pandas as pd
import numpy as np
from google.colab import drive

# allow access to google drive
drive.mount('/content/drive')

# read in data
generators = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/generators_for_expansion.csv", index_col = "G")
generators.index.name = "Source"

demand = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/demand_for_expansion.csv", index_col = "Hour")
demand = demand["Demand"] # coerce from datatype pd.DataFrame to pd.Series (important later on)

# create environment with Gurobi academic license
params = {
"WLSACCESSID": ,
"WLSSECRET": ,
"LICENSEID": ,
}
env = gp.Env(params=params)

## Optimization only considering thermal power plants (from class)

In [ ]:
# parameters
NSEcost = 9000

# Create the model within the Gurobi environment
model = gp.Model('Capacity Expansion',env=env)

# create dataframes to contain gurobi variables, gurobi constrants, and parameters
## by source
df_by_src = generators.loc[:, ['FixedCost', 'VarCost']].copy()
df_by_src.loc["NSE", 'FixedCost'] = 0 # consider NSE a generator with no fixed cost
df_by_src.loc["NSE", 'VarCost'] = NSEcost # variable cost of NSE is NSEcost
df_by_src.drop(["Wind","Solar"],axis=0,inplace=True) # drop wind and solar for now

## by source and hour
### create multi-indexed dataframe for each decision variable index
list_of_indices = [
    list(df_by_src.index),
    list(demand.index) # hours
]

multi_idx = pd.MultiIndex.from_product(list_of_indices, names=["Source", "Hour"])

df_by_hr = pd.DataFrame(index = multi_idx)

# default bounds are [0,infinity) unless otherwise specified, so do not need to specify bounds
GEN = df_by_hr.gppd.add_vars(model, name = "gen_MWh", vtype=GRB.CONTINUOUS)
CAP = df_by_src.gppd.add_vars(model, name = "cap_MW", vtype=GRB.CONTINUOUS)

# non-negativity constraints assumed, just add demand and capacity constraints
# pandas.Dataframe.join documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html
c1_capacity = gppd.add_constrs(
    model, GEN['gen_MWh'], GRB.LESS_EQUAL, GEN.join(CAP['cap_MW'])['cap_MW'], name="c1_capacity"
)

# pandas.Dataframe.groupby documentation:  https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
c2_demand = gppd.add_constrs(
    model, GEN['gen_MWh'].groupby(level = "Hour").sum(), # this sums all power generated from all sources each hour
    GRB.EQUAL, demand, name="c2_demand" # demand needed to be a pd.Series datatype here
)

# objective function
gen_costs = (df_by_src["VarCost"] * GEN["gen_MWh"]).agg(gp.quicksum) # includes NSE penalty
fixed_costs = (df_by_src["FixedCost"] * CAP['cap_MW']).agg(gp.quicksum)

model.setObjective(gen_costs + fixed_costs)

model.write('capacity_expansion.lp')
model.optimize()

In [ ]:
# process capacity expansion results
cap = CAP.cap_MW.gppd.x
cap_share = cap/np.max(demand)*100 # percent of max demand (we want to be sure we can meet this)
cap_share.name = "cap_share_of_peak_demand" # name the pandas series (this will become the column header when we
                                            # add it to our results dataframe)

# process energy generation results
gen = GEN.gen_MWh.gppd.x.groupby(level = "Source").sum() # sum power generated by source
genshare = gen / np.sum(demand)*100 # compute the fraction of demand met by each source
genshare.name = "gen_share_of_total_demand"
gen = gen / 1000 # convert from MWh to GWh
gen.name = "Gen_GWh"

# Pandas concat documentation: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
df_results = pd.concat([cap,cap_share, gen, genshare], axis = 1) # axis = 1 because we are combining columns rather than rows

df_results.loc['SUM', :] = df_results.sum() # add a row equal to the sum of each column as a check

df_results

## 2. Optimization with renewables

Create a new model, model2 that includes renewables. To do this, add renewables back to df_by_src and df_by_hr. Also include a column in df_by_hr for the capacity factors.
$\color{red}{\text{The code below is complete.}}$

In [ ]:
# create dataframes to contain gurobi variables, gurobi constrants, and parameters
## by source
df_by_src = generators.loc[:, ['FixedCost', 'VarCost']].copy()
df_by_src.loc["NSE", 'FixedCost'] = 0 # consider NSE a generator with no fixed cost
df_by_src.loc["NSE", 'VarCost'] = NSEcost # variable cost of NSE is NSEcost

df_by_src

In [ ]:
# Read in capacity factors for wind and solar
capacity_factors = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/wind_solar_for_expansion.csv", index_col = "Hour")
capacity_factors.head()

### re-organize wide to long using pd.DataFrame.stack()
# documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html
capacity_factors = capacity_factors.T.stack()
capacity_factors.index.names = ["Source", "Hour"]
capacity_factors.name = "capfac"
capacity_factors

# redefine df_by_hr so that it includes Wind and Solar
## by source and hour
### create multi-indexed dataframe for each decision variable index
list_of_indices = [
    list(df_by_src.index),
    list(demand.index) # hours
]

multi_idx = pd.MultiIndex.from_product(list_of_indices, names=["Source", "Hour"])
# create dataframe
df_by_hr = pd.DataFrame(index = multi_idx)
# add the capacity factors
df_by_hr = df_by_hr.join(capacity_factors)

# fillna(1) puts a capacity factor of 1 for generators that have no capacity factor data (generators other than wind and solar)
df_by_hr = df_by_hr.fillna(1)

df_by_hr

In [ ]:
# Create the model within the Gurobi environment
model2 = gp.Model('Capacity Expansion with Renewables',env=env)

$\color{red}{\text{Edit the code below to adapt the decision variables, constraints, and objective function with renewables included.}}$

In [ ]:
# decision variables (defined as GEN and CAP)


# constraints


# objective function

model2.setObjective(gen_costs + fixed_costs)
model2.write('capacity_expansion_renewables.lp')
model2.optimize()

$\color{red}{\text{Print the results. The code below is complete.}}$

In [ ]:
# process capacity expansion results
cap = CAP.cap_MW.gppd.x
cap_share = cap/np.max(demand)*100 # percent of max demand (we want to be sure we can meet this)
cap_share.name = "cap_share_of_peak_demand" # name the pandas series (this will become the column header when we
                                            # add it to our results dataframe)

# process energy generation results
gen = GEN.gen_MWh.gppd.x.groupby(level = "Source").sum() # sum power generated by source
genshare = gen / np.sum(demand)*100 # compute the fraction of demand met by each source
genshare.name = "gen_share_of_total_demand"
gen = gen / 1000 # convert from MWh to GWh
gen.name = "Gen_GWh"

# Pandas concat documentation: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
df_results2 = pd.concat([cap,cap_share, gen, genshare], axis = 1) # axis = 1 because we are combining columns rather than rows

df_results2.loc['SUM', :] = df_results2.sum() # add a row equal to the sum of each column as a check

df_results2

## 3. Optimization considering renewables and retirements

Add information on the old generators to the data frame.
$\color{red}{\text{The code below is complete.}}$

In [ ]:
# Add data on old generators to dataframe
generators.loc["Old_CCGT",:] = ["Existing CCGT",0,40000,5,7.5,4,0,0,0,0,40000,30]
generators.loc["Old_CT",:] = ["Existing CT",0,30000,11,11.0,4,0,0,0,0,30000,55]

# Set installed capacity for existing CCGTs:
ExistingCap_CCGT = 1260 # Approximate actual existing capacity in SDGE
ExistingCap_CT = 925 # Approximate actual existing capacity in SDGE
# Add new column to generators Data Frame
generators["ExistingCap"] = [0,0,0,0,0,0,ExistingCap_CCGT,ExistingCap_CT]
generators

Create a new model, model3, that includes old generators. To do this, create separate data frames by src and by hr of both old and new generators. $\color{red}{\text{The code below is complete.}}$

In [ ]:
# create dataframes to contain gurobi variables, gurobi constrants, and parameters
## by source
df_by_src = generators.loc[:, ['FixedCost', 'VarCost','ExistingCap']].copy()
df_by_src.loc["NSE", 'FixedCost'] = 0 # consider NSE a generator with no fixed cost
df_by_src.loc["NSE", 'VarCost'] = NSEcost # variable cost of NSE is NSEcost
df_by_src.loc["NSE", 'ExistingCap'] = 0 # variable cost of NSE is NSEcost

df_by_src

In [ ]:
# separate old and new sources into separate data frames since different constraints apply to each
df_by_oldsrc = df_by_src.loc[["Old_CCGT","Old_CT"]]
df_by_oldsrc

In [ ]:
# separate old and new sources into separate data frames since different constraints apply to each
df_by_newsrc = df_by_src.loc[["Geo","Coal","CCGT","CT","Wind","Solar","NSE"]]
df_by_newsrc

In [ ]:
# redefine df_by_hr so that it includes Wind and Solar
## by source and hour
### create multi-indexed dataframe for each decision variable index
list_of_indices = [
    list(df_by_src.index),
    list(demand.index) # hours
]

multi_idx = pd.MultiIndex.from_product(list_of_indices, names=["Source", "Hour"])
# create dataframe
df_by_hr = pd.DataFrame(index = multi_idx)
# add the capacity factors
df_by_hr = df_by_hr.join(capacity_factors)

# fillna(1) puts a capacity factor of 1 for generators that have no capacity factor data (generators other than wind and solar)
df_by_hr = df_by_hr.fillna(1)

df_by_hr

In [ ]:
# separate old and new sources
idx = pd.IndexSlice
df_by_hr_oldsrc = df_by_hr.loc[idx[["Old_CCGT","Old_CT"],:],:]
df_by_hr_oldsrc

In [ ]:
# separate old and new sources
idx = pd.IndexSlice
df_by_hr_newsrc = df_by_hr.loc[idx[["Geo","Coal","CCGT","CT","Wind","Solar","NSE"],:],:]
df_by_hr_newsrc

In [ ]:
# Create the model within the Gurobi environment
model3 = gp.Model('Capacity Expansion and Retirement',env=env)

$\color{red}{\text{Edit the code below to adapt the decision variables, constraints, and objective function with renewables and old generators included.}}$  
$\color{red}{\text{Capacity decisions apply to df_by_newsrc. Retirement decisions apply to df_by_old_src.}}$  
$\color{red}{\text{Generation decisions apply to df_by_hr_newsrc and df_by_hr_oldsrc.}}$

In [ ]:
# decision variables (defined as GEN_OLD, GEN_NEW, CAP and RET)


# constraints


# objective function


model3.setObjective(gen_old_costs + gen_new_costs + fixed_old_costs + fixed_new_costs)
model3.write('capacity_expansion_retirement.lp')
model3.optimize()

$\color{red}{\text{Print the results. The code below is complete.}}$

In [ ]:
# process capacity expansion results
cap_new = CAP.cap_MW.gppd.x
cap_new = cap_new.to_frame()
ret_old = RET.ret_MW.gppd.x
ret_old = ret_old.to_frame()

cap_old = df_by_oldsrc['ExistingCap'] - ret_old["ret_MW"]
cap_old = cap_old.to_frame()
cap_old.rename(columns={0:"cap_MW"},inplace=True)
cap = pd.concat((cap_new,cap_old),0)

cap_share = cap/np.max(demand)*100 # percent of max demand (we want to be sure we can meet this)
cap_share.rename(columns={"cap_MW":"cap_share_of_peak_demand"},inplace=True) # name the pandas series (this will become the column header when we
                                                                              # add it to our results dataframe)

# process energy generation results
gen_old = GEN_OLD.gen_MWh.gppd.x.groupby(level = "Source").sum()
gen_new = GEN_NEW.gen_MWh.gppd.x.groupby(level = "Source").sum()
gen = pd.concat((gen_old,gen_new),0)

genshare = gen / np.sum(demand)*100 # compute the fraction of demand met by each source
genshare.name = "gen_share_of_total_demand"
gen = gen / 1000 # convert from MWh to GWh
gen.name = "Gen_GWh"

# add column for retirements
ret_new = pd.DataFrame(index=cap_new.index,columns=["ret_MW"],data=0.0)
ret = pd.concat((ret_new,ret_old),0)

# Pandas concat documentation: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
df_results3 = pd.concat([cap, cap_share, gen, genshare, ret], axis = 1) # axis = 1 because we are combining columns rather than rows

df_results3.loc['SUM', :] = df_results3.sum() # add a row equal to the sum of each column as a check

df_results3